# 簡易レコメンデーションシステムの実装
人工データおよびMovieLensのデータについて、簡単なレコメンデーションシステムを作り実験する。

## 準備
[MovieLensのホームページ](https://grouplens.org/datasets/movielens/)からファイルml-latest-small.zipをダウンロードして解凍する。ファイルratings.csvとmovies.csvを使うのでこのノードブックと同じディレクトリに置いておくこと。

共通のインポート：

In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors, DistanceMetric
from scipy.sparse import lil_matrix

## 人工データによる実験
まずは動作原理を理解するために、人工データによる実験を行う

まずは疎行列型（lil_matrix）を使って人工データの準備をする。

In [39]:
data = lil_matrix((5,5))
data[0,0]=5; data[0,2]=3; data[0,3]=5
data[1,0]=4; data[1,1]=2; data[1,2]=3; data[1,4]=4
data[2,1]=1; data[2,3]=2; data[2,4]=2
data[3,0]=2; data[3,2]=4; data[3,4]=2
data[4,0]=1; data[4,1]=5; data[4,3]=2

In [40]:
pd.DataFrame(data.todense())

,0,1,2,3,4
0,5.0,0.0,3.0,5.0,0.0
1,4.0,2.0,3.0,0.0,4.0
2,0.0,1.0,0.0,2.0,2.0
3,2.0,0.0,4.0,0.0,2.0
4,1.0,5.0,0.0,2.0,0.0


kNNを使った学習を行う。`fit`の引数は、行列（疎行列または蜜行列）で、各行が点（ベクトル）として扱われる。

In [41]:
n_neighbors=3
model = NearestNeighbors(n_neighbors=n_neighbors)
model.fit(data)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=3, p=2, radius=1.0)

計算結果を2つの変数に格納する。`indices`にはそれぞれの点がどの点に近いかという、近接する点のインデクスが入り、`distance`にはそれぞれの点への距離が入る。

In [42]:
distances, indices = model.kneighbors()

In [43]:
distances

array([[6.244998  , 6.78232998, 6.92820323],
       [3.60555128, 5.83095189, 6.78232998],
       [4.58257569, 5.        , 5.83095189],
       [3.60555128, 5.        , 6.244998  ],
       [4.58257569, 6.8556546 , 7.07106781]])

In [44]:
indices

array([[3, 1, 2],
       [3, 2, 0],
       [4, 3, 1],
       [1, 2, 0],
       [2, 1, 3]], dtype=int64)

この場合、インデクス0の点に近いのは、近い順に3, 1, 2の点であり、0の点から3, 1, 2の点への距離は、それぞれ6.244998, 6.78232998, 6.92820323である。

`data.T`とすることで`data`の転置行列をとることができる。

In [46]:
pd.DataFrame(data.T.todense())

,0,1,2,3,4
0,5.0,4.0,0.0,2.0,1.0
1,0.0,2.0,1.0,0.0,5.0
2,3.0,3.0,0.0,4.0,0.0
3,5.0,0.0,2.0,0.0,2.0
4,0.0,4.0,2.0,2.0,0.0


転置についてのkNNを計算すると、今度は各列を点と見たときのkNNが計算できる。

In [9]:
n_neighbors=3
model = NearestNeighbors(n_neighbors=n_neighbors)
model.fit(data.T)
distances, indices = model.kneighbors()

In [10]:
distances

array([[3.16227766, 5.        , 5.47722558],
       [5.83095189, 6.244998  , 7.07106781],
       [3.16227766, 4.24264069, 6.08276253],
       [5.        , 6.08276253, 6.244998  ],
       [4.24264069, 5.47722558, 5.83095189]])

In [11]:
indices

array([[2, 3, 4],
       [4, 3, 0],
       [0, 4, 3],
       [0, 2, 1],
       [2, 0, 1]], dtype=int64)

## MovieLensデータによる実験

まずはレイティングデータを読み込む。

In [52]:
ratings = pd.read_csv("ratings.csv")
ratings.head(20)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
9,1,157,5.0,964984100


次に映画データを読み込む。

In [51]:
movies = pd.read_csv("movies.csv")
movies.head(50)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


movieIdは飛び飛びの番号になっているので、movieIdとインデクス（一番左の列）との対応をできるようにする。

In [60]:
movies["idx"] = movies.index
movies_idx2id = movies["movieId"] # インデクスからmovieIdへの変換
movies_id2idx = movies[["movieId", "idx"]].set_index("movieId").iloc[:,0] # movieIdからインデクスへの変換
movies_idx2title = movies["title"] # インデクスから映画名への変換

ユーザを行、映画を列とする行列を作成。まず空の疎行列を作ってからデータを詰めていく。

In [62]:
size = (ratings["userId"].max(), movies.index.max()+1)
data = lil_matrix(tuple(size))
data.shape

(610, 9742)

In [63]:
for u,m,r in ratings[["userId", "movieId", "rating"]].values:
    data[u-1, movies_id2idx[m]] = r

### アイテムベースのレコメンデーション
まず学習させる。

In [66]:
n_neighbors=10
model = NearestNeighbors(n_neighbors=n_neighbors)
model.fit(data.T.tocsr())
distances, indices = model.kneighbors()

タイトルに文字列`"Star Wars"`を含むものの一覧を表示。

In [67]:
b=movies["title"].apply(lambda x: "Star Wars" in x)
movies[b]

,movieId,title,genres,idx
224,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,224
898,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi,898
911,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Sci-Fi,911
1979,2628,Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Sci-Fi,1979
3832,5378,Star Wars: Episode II - Attack of the Clones (...,Action|Adventure|Sci-Fi|IMAX,3832
5896,33493,Star Wars: Episode III - Revenge of the Sith (...,Action|Adventure|Sci-Fi,5896
6823,61160,Star Wars: The Clone Wars (2008),Action|Adventure|Animation|Sci-Fi,6823
7367,79006,Empire of Dreams: The Story of the 'Star Wars'...,Documentary,7367
8683,122886,Star Wars: Episode VII - The Force Awakens (2015),Action|Adventure|Fantasy|Sci-Fi|IMAX,8683
8908,135216,The Star Wars Holiday Special (1978),Adventure|Children|Comedy|Sci-Fi,8908


movieId:2628「Star Wars: Episode I - The Phantom Menace (1999)」に近いもののタイトルを表示

In [68]:
movies_idx2title[indices[movies_id2idx[2628]]]

3832    Star Wars: Episode II - Attack of the Clones (...
2248                                       RoboCop (1987)
5896    Star Wars: Episode III - Revenge of the Sith (...
1522                      Honey, I Shrunk the Kids (1989)
2193                                  Total Recall (1990)
1567                                          Tron (1982)
1164                Lost World: Jurassic Park, The (1997)
1060                                Batman Returns (1992)
2029                                Wild Wild West (1999)
1171                                       Con Air (1997)
Name: title, dtype: object